In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
WORKING_PATH = '/content/drive/MyDrive/Colab Notebooks/AgentesIA'
%cd {WORKING_PATH}

import sys
sys.path.append(WORKING_PATH)

/content/drive/MyDrive/Colab Notebooks/AgentesIA


In [3]:
import torch
if torch.cuda.is_available():
    print("¡GPU detectada!:", torch.cuda.get_device_name(0))
else:
    print("No se encontró GPU. Se usará CPU.")

No se encontró GPU. Se usará CPU.


In [4]:
!pip install transformers torch langchain langchain_community langchain_openai PyPDF2 faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 42.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 61.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 68.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.3/55.3 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.

In [5]:
import os
import numpy as np
from PyPDF2 import PdfReader
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv, find_dotenv

In [8]:
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ.get("OPENAI_API_KEY")
#print("OPENAI_API_KEY:", openai_api_key)

In [9]:
lm_model = ChatOpenAI(model="gpt-3.5-turbo-0125")
embedder = HuggingFaceBgeEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
data_directory = "/content/drive/MyDrive/Colab Notebooks/AgentesIA/data"

<ipython-input-9-b9652c90331e>:2: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedder = HuggingFaceBgeEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to a

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.12k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
def read_pdf(pdf_path):
    pdf_reader = PdfReader(pdf_path)
    text = ""
    for page in pdf_reader.pages:
        page_text = page.extract_text()
        if page_text:
            text += page_text + "\n"
    return text

In [11]:
def cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

In [12]:
def find_most_relevant_document(question, document_embeddings):
    query_embedding = embedder.embed_query(question)
    best_similarity = -1
    best_doc = None
    for doc_name, doc_info in document_embeddings.items():
        sim = cosine_similarity(query_embedding, doc_info["embedding"])
        if sim > best_similarity:
            best_similarity = sim
            best_doc = doc_name
    return best_doc

In [13]:
pdf_files = [f for f in os.listdir(data_directory) if f.endswith('.pdf')]

document_embeddings = {}
for pdf_file in pdf_files:
    pdf_path = os.path.join(data_directory, pdf_file)
    pdf_text = read_pdf(pdf_path)
    embedding = embedder.embed_documents([pdf_text])[0]
    document_embeddings[pdf_file] = {
        "text": pdf_text,
        "embedding": embedding
    }

In [14]:
while True:
    user_question = input("Introduce tu pregunta sobre los documentos PDF (o escribe 'salir' para terminar): ")
    if user_question.lower() in ['salir', 'exit', 'quit']:
        break

    most_relevant_document = find_most_relevant_document(user_question, document_embeddings)
    relevant_document_text = document_embeddings[most_relevant_document]["text"]

    prompt = f"Contexto: {relevant_document_text}\n\nPregunta: {user_question}\n\nRespuesta:"

    response = lm_model(prompt)
    print(f"Respuesta: {response.content}\n")


Introduce tu pregunta sobre los documentos PDF (o escribe 'salir' para terminar): servicios training2works


<ipython-input-14-0c04d5e34344>:11: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = lm_model(prompt)


Respuesta: Training2Work ofrece servicios de formación en servicios digitales para estudiantes, prácticas en empresas de consultoría donde aplican sus conocimientos y búsqueda de trabajo en empresas finales. Su objetivo es ofrecer formación orientada a la salida profesional en el mundo digital y diferenciarse de la competencia ofreciendo perfiles Junior con soporte Senior a un coste económico pero de alta calidad. Su público objetivo son personas jóvenes con capacidad de aprendizaje en el mundo digital y empresas que necesitan perfiles digitales de calidad. Buscan resolver la necesidad de inserción laboral en el mundo digital y ofrecer perfiles cualificados a empresas a un coste rentable. Utilizan un tono formal pero revolucionario en su comunicación y sus estrategias de marketing se basan en canales digitales como SEO, redes sociales y formación online. Su éxito se basa en ofrecer un buen servicio, con testimonios de clientes satisfechos como prueba.

Introduce tu pregunta sobre los d